In [1]:
from enum import Enum
class Suit(Enum):
  HEART = "H"
  DIAMOND = "D"
  CLUB = "C"
  SPADE = "S"
  

In [2]:
class Rank(Enum):
  ACE = "A"
  TWO = "2"
  THREE = "3"
  FOUR ="4"
  FIVE ="5"
  SIX = "6"
  SEVEN = "7"
  EIGHT ="8"
  NINE ="9"
  TEN = "10"
  JACK ="J"
  QUEEN ="Q"
  KING ="K"

In [3]:
class Card:
  def __init__(self, suit, rank, is_showing = True):
    self.suit = suit
    self.rank = rank
    self.is_showing = is_showing

  def __str__(self):
    return str(self.rank.value) + str(self.suit.value)

  def is_face(self):
    if self.rank in set([Rank.TEN, Rank.JACK, Rank.QUEEN, Rank.KING, Rank.ACE]):
      return True
    else:
      return False

  def is_low(self):
    if self.rank in set([Rank.TWO, Rank.THREE, Rank.FOUR, Rank.FIVE, Rank.SIX]):
      return True
    else:
      return False


In [4]:
import random
class Shoe:
  def __init__(self, num_decks = 6):
    self.shoe = []
    self.discard = []
    for i in range(num_decks):
      for suit in [Suit.HEART, Suit.DIAMOND, Suit.CLUB, Suit.SPADE]:
        for rank in [Rank.ACE, Rank.TWO, Rank.THREE, Rank.FOUR, Rank.FIVE, Rank.SIX, Rank.SEVEN, Rank.EIGHT, Rank.NINE, Rank.TEN, Rank.JACK, Rank.QUEEN, Rank.KING]:
          self.shoe.append(Card(suit, rank))

  def deal(self, num_cards):
    popped_cards = [] 
    for idx in range(num_cards):
      if not self.shoe:
        self.reshuffle()
      popped_cards.append(self.shoe.pop())
    return popped_cards

  def shuffle(self):
    random.shuffle(self.shoe)

  def __str__(self):
    return ",".join(map(str,self.shoe))
         
  def add_to_discard(self,used_cards):
    self.discard.extend(used_cards)

  def reshuffle(self):
    self.shoe.extend(self.discard)
    self.discard = []
    self.shuffle()

In [5]:
class Hand:
  def __init__(self):
    self.cards = []
  
  def add_cards(self, new_cards):
    self.cards.extend(new_cards)

  def __str__(self):
    return ",".join(map(str,self.cards))

  def get_total(self,ace_value=1):
    total = 0
    for card in self.cards:
      if card.rank == Rank.ACE:
        total += ace_value
      elif card.rank == Rank.JACK or card.rank == Rank.QUEEN or card.rank == Rank.KING:
        total += 10
      else:
        total = total + int(card.rank.value)
    return total

  def is_bust(self):
    if self.get_total() > 21:
      return True
    else: 
      return False
      
  def is_blackjack(self):
    if len(self.cards) != 2:
      return False
    if len(self.cards) == 2 and self.get_total() == 11:
      return True
    else:
      return False
      

In [6]:
class Action(Enum):
  HIT = 0
  STAY = 1

In [7]:
from enum import Enum
class GameStatus(Enum):
  PLAYER_WIN = 0
  DEALER_WIN = 1
  TIE = 2
  ONGOING = 3

In [8]:
class Blackjack:
  def __init__(self):
    self.shoe = Shoe()
    self.shoe.shuffle()
    
  def start_deal(self):
    self.dealer_hand = Hand()
    self.player_hand = Hand()
    self.player_hand.add_cards(self.shoe.deal(2))
    self.dealer_hand.add_cards(self.shoe.deal(2))
    self.dealer_hand.cards[0].is_showing = False

    if self.dealer_hand.is_blackjack() and self.player_hand.is_blackjack():
      return (self.player_hand, self.dealer_hand, GameStatus.TIE)
    elif self.dealer_hand.is_blackjack():
      return (self.player_hand, self.dealer_hand, GameStatus.DEALER_WIN)
    elif self.player_hand.is_blackjack():
      return (self.player_hand, self.dealer_hand, GameStatus.PLAYER_WIN)
    return (self.player_hand, self.dealer_hand, GameStatus.ONGOING)

  def collect_hands(self, player_hand,dealer_hand):
    self.shoe.add_to_discard(player_hand.cards)
    self.shoe.add_to_discard(dealer_hand.cards)
    
  def next_action(self, action):
    game_status = GameStatus.ONGOING
    if action == Action.STAY:
      while dealer_hand.get_total() < 17:
        new_card = self.shoe.deal(1)
        self.dealer_hand.add_cards(new_card)
      if dealer_hand.is_bust():
        game_status = GameStatus.PLAYER_WIN
      else: 
        if self.dealer_hand.get_total() > self.player_hand.get_total():
          game_status = GameStatus.DEALER_WIN
        elif self.dealer_hand.get_total() == self.player_hand.get_total():
          game_status = GameStatus.TIE
        else:
          game_status = GameStatus.PLAYER_WIN
    elif action == Action.HIT:
      new_card = self.shoe.deal(1)
      self.player_hand.add_cards(new_card)
      if self.player_hand.is_bust():
        game_status = GameStatus.DEALER_WIN
    return (self.player_hand, self.dealer_hand, game_status)

In [9]:
class StayStrategy:
  def choose_action(self,player_hand, dealer_hand):
    return Action.STAY
  

In [10]:
import random 
class RandomStrategy:
  def choose_action(self,player_hand, dealer_hand):
    if random.random() == 0:
      return Action.STAY
    else:
      return Action.HIT

In [11]:
class SafeStrategy:
  def choose_action(self,player_hand, dealer_hand):
    if player_hand.get_total() >= 17 and player_hand.get_total(ace_value=11) >= 17:
      return Action.STAY
    else: 
      return Action.HIT

In [12]:
class SimpleBetter:
  def track_hands(self,player_hand, dealer_hand):
    return None

  def betting_amount(self,total_balance):
    bet_total = total_balance/1000
    if total_balance < 25:
      bet_total = 0
    elif bet_total < 25:
      bet_total = 25

    return bet_total

In [13]:
"""class CardCounterV1:
  def __init__(self):
    self.cards_seen = 0 
    self.face_cards_seen = 0
    self.low_cards_seen = 0
    self.num_points = 0

  def count_cards(self,cards):
    for card in cards:
      self.cards_seen += 1
      if card.is_face():
        self.face_cards_seen +=1
        self.num_points -=1
      elif card.is_low():
        self.low_cards_seen +=1
        self.num_points +=1

  def track_hands(self,player_hand, dealer_hand):
    self.count_cards(player_hand.cards)
    self.count_cards(dealer_hand.cards)

  def betting_amount(self,total_balance):
    bet_total = total_balance/1000 * -1*(self.num_points-1)
    true_count = self.num_points
    if total_balance < 25:
      bet_total = 0
    elif bet_total < 25:
      bet_total = 25

    return bet_total"""
    
  


In [14]:

num_sims = 500
num_rounds = 100
strategies = [RandomStrategy(), StayStrategy(), SafeStrategy()]

better = SimpleBetter()

for strategy in strategies:
  num_player_wins = 0
  num_ties = 0
  num_trials = 0
  total_earnings = 0

  for i in range(0, num_sims):
    bank_balance = 10000
    blackjackGame = Blackjack()
    for j in range(0, num_rounds):
      bet_amount = better.betting_amount(bank_balance)
      if bet_amount == 0:
        break

      (player_hand, dealer_hand, game_status) = blackjackGame.start_deal()

      while game_status == GameStatus.ONGOING:
        (player_hand, dealer_hand, game_status) = blackjackGame.next_action(strategy.choose_action(player_hand, dealer_hand))
      if game_status == GameStatus.PLAYER_WIN:
        num_player_wins = num_player_wins + 1
        bank_balance += bet_amount
      elif game_status == GameStatus.TIE:
        num_ties = num_ties + 1
      else:
        bank_balance -= bet_amount

      
      blackjackGame.collect_hands(player_hand, dealer_hand)
      better.track_hands(player_hand, dealer_hand)
      num_trials += 1
      if bank_balance <= 0:
        break

    total_earnings = total_earnings + bank_balance 

  pct_play_wins = 100*num_player_wins/num_trials
  pct_ties = 100*num_ties/num_trials
  pct_play_losses = 100 - pct_play_wins - pct_ties
  ave_earnings = total_earnings/num_sims
  print("{} - Wins: {:.2f}%, Ties: {:.2f}%, Losses: {:.2f}, Ave. earnings: {}".format(type(strategy).__name__, pct_play_wins, pct_ties, pct_play_losses, ave_earnings))
    

RandomStrategy - Wins: 8.82%, Ties: 0.91%, Losses: 90.27, Ave. earnings: 7963.65
StayStrategy - Wins: 41.06%, Ties: 4.64%, Losses: 54.30, Ave. earnings: 9668.95
SafeStrategy - Wins: 41.35%, Ties: 8.91%, Losses: 49.74, Ave. earnings: 9790.25


Todo:
* new class in bj game "payout" 
* update blj to support doubling
*  update rules: https://www.quora.com/How-can-people-win-in-a-blackjack-without-counting 
*  update betting strategy: https://www.countingedge.com/card-counting/true-count/ (https://www.888casino.com/blog/blackjack-tips/the-win-rate-of-the-average-blackjack-card-counter)
* fix counting strat, buggy/not working
*input game?






In [19]:
"""strategy =  RandomStrategy()
blackjackGame = Blackjack()
print("cards in shoe: {}".format(len(blackjackGame.shoe.shoe)))
(player_hand, dealer_hand, game_status) = blackjackGame.start_deal()
print("Dealer hand: " + str(dealer_hand))
print("Player hand: " + str(player_hand))
print("Dealer hand total: " + str(dealer_hand.get_total()))
print("Player hand total: " + str(player_hand.get_total()))
print("Game status: " + str(game_status))

while game_status == GameStatus.ONGOING:
    action = strategy.choose_action(player_hand, dealer_hand)
    print(action)
    (player_hand, dealer_hand, game_status) = blackjackGame.next_action(action)
    print("Dealer hand: " + str(dealer_hand))
    print("Player hand: " + str(player_hand))
    print("Dealer hand total: " + str(dealer_hand.get_total()))
    print("Player hand total: " + str(player_hand.get_total()))
    print("Game status: " + str(game_status))

blackjackGame.collect_hands(player_hand, dealer_hand)
print("# of cards in discard pile: {}".format(len(blackjackGame.shoe.discard)))
print("# of cards left in shoe: {}".format(len(blackjackGame.shoe.shoe)))"""




'strategy =  RandomStrategy()\nblackjackGame = Blackjack()\nprint("cards in shoe: {}".format(len(blackjackGame.shoe.shoe)))\n(player_hand, dealer_hand, game_status) = blackjackGame.start_deal()\nprint("Dealer hand: " + str(dealer_hand))\nprint("Player hand: " + str(player_hand))\nprint("Dealer hand total: " + str(dealer_hand.get_total()))\nprint("Player hand total: " + str(player_hand.get_total()))\nprint("Game status: " + str(game_status))\n\nwhile game_status == GameStatus.ONGOING:\n    action = strategy.choose_action(player_hand, dealer_hand)\n    print(action)\n    (player_hand, dealer_hand, game_status) = blackjackGame.next_action(action)\n    print("Dealer hand: " + str(dealer_hand))\n    print("Player hand: " + str(player_hand))\n    print("Dealer hand total: " + str(dealer_hand.get_total()))\n    print("Player hand total: " + str(player_hand.get_total()))\n    print("Game status: " + str(game_status))\n\nblackjackGame.collect_hands(player_hand, dealer_hand)\nprint("# of cards i

In [ ]:
list(range(20))